# 🚀 Day 3: GPU Memory Fundamentals

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sdodlapati3/cuda-lab/blob/main/learning-path/week-01/day-3-memory-basics.ipynb)

## Learning Philosophy

> **CUDA C++ First, Python/Numba as Optional Backup**

This notebook shows:
1. **CUDA C++ code** - The PRIMARY implementation you should learn
2. **Python/Numba code** - OPTIONAL for quick interactive testing in Colab

> **Note:** If running on Google Colab, go to `Runtime → Change runtime type → T4 GPU` before starting!

---

In [ ]:
# Verify CUDA is available
!nvcc --version
!nvidia-smi --query-gpu=name,memory.total,compute_cap --format=csv

---

# Day 3: GPU Memory Fundamentals

Memory management is **the most important skill** in CUDA programming. Today you'll learn:
- GPU memory hierarchy
- Explicit memory management (`cudaMalloc`, `cudaMemcpy`, `cudaFree`)
- Unified/Managed memory (`cudaMallocManaged`)
- Pinned vs pageable memory for transfers
- Performance implications of each approach

---

## 1. The Memory Hierarchy

```
┌─────────────────────────────────────────────────────────────────┐
│                        GPU MEMORY HIERARCHY                      │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│   Registers (per thread)     ← Fastest (~0 latency)             │
│   ├── ~255 32-bit registers per thread                          │
│   └── Private to each thread                                    │
│                                                                  │
│   Shared Memory (per block)  ← Very fast (~5 cycles)            │
│   ├── 48-164 KB per SM                                          │
│   └── Shared between threads in same block                      │
│                                                                  │
│   L1 Cache / Texture Cache   ← Fast (~30 cycles)                │
│   └── Automatic caching                                         │
│                                                                  │
│   L2 Cache                   ← Medium (~200 cycles)             │
│   └── Shared across all SMs                                     │
│                                                                  │
│   Global Memory (VRAM)       ← Slow (~400+ cycles)              │
│   ├── 4-80 GB (main GPU memory)                                 │
│   └── Accessible by all threads                                 │
│                                                                  │
├─────────────────────────────────────────────────────────────────┤
│   Host Memory (System RAM)   ← Very slow (PCIe transfer)        │
│   └── Must be copied to GPU before use                          │
└─────────────────────────────────────────────────────────────────┘
```

## 2. Explicit Memory Management

The traditional CUDA workflow requires **explicit** memory transfers:

```
1. Allocate memory on GPU     (cudaMalloc)
2. Copy data CPU → GPU        (cudaMemcpy H2D)
3. Launch kernel
4. Copy results GPU → CPU     (cudaMemcpy D2H)
5. Free GPU memory            (cudaFree)
```

Let's implement the classic **SAXPY** operation: `y = a*x + y`

In [ ]:
%%writefile explicit_memory.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <time.h>

// SAXPY kernel: y = a*x + y
__global__ void saxpy(float a, float* x, float* y, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        y[idx] = a * x[idx] + y[idx];
    }
}

void explicitMemoryWorkflow(int n, float a) {
    printf("\n=== Explicit Memory Workflow (N = %d) ===\n", n);
    printf("---------------------------------------------------\n");
    
    size_t size = n * sizeof(float);
    
    // Step 1: Allocate host memory
    float *h_x = (float*)malloc(size);
    float *h_y = (float*)malloc(size);
    float *h_result = (float*)malloc(size);
    
    // Initialize host data
    srand(42);
    for (int i = 0; i < n; i++) {
        h_x[i] = (float)rand() / RAND_MAX;
        h_y[i] = (float)rand() / RAND_MAX;
        h_result[i] = h_y[i];  // Copy for verification
    }
    
    // Step 2: Allocate device memory
    float *d_x, *d_y;
    cudaMalloc(&d_x, size);
    cudaMalloc(&d_y, size);
    
    // Create CUDA events for timing
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    // Step 3: Copy Host → Device (H2D)
    cudaEventRecord(start);
    cudaMemcpy(d_x, h_x, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_y, h_result, size, cudaMemcpyHostToDevice);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float h2dTime;
    cudaEventElapsedTime(&h2dTime, start, stop);
    printf("  Host→Device transfer: %.3f ms\n", h2dTime);
    
    // Step 4: Launch kernel
    int threadsPerBlock = 256;
    int blocksPerGrid = (n + threadsPerBlock - 1) / threadsPerBlock;
    
    cudaEventRecord(start);
    saxpy<<<blocksPerGrid, threadsPerBlock>>>(a, d_x, d_y, n);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float kernelTime;
    cudaEventElapsedTime(&kernelTime, start, stop);
    printf("  Kernel execution:      %.3f ms\n", kernelTime);
    
    // Step 5: Copy Device → Host (D2H)
    cudaEventRecord(start);
    cudaMemcpy(h_result, d_y, size, cudaMemcpyDeviceToHost);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float d2hTime;
    cudaEventElapsedTime(&d2hTime, start, stop);
    printf("  Device→Host transfer:  %.3f ms\n", d2hTime);
    
    printf("  -----------------------------------\n");
    printf("  Total time:            %.3f ms\n", h2dTime + kernelTime + d2hTime);
    
    // Verify results
    bool correct = true;
    for (int i = 0; i < n && correct; i++) {
        float expected = a * h_x[i] + h_y[i];
        if (fabsf(h_result[i] - expected) > 1e-5) {
            printf("  Mismatch at %d: %f vs %f\n", i, h_result[i], expected);
            correct = false;
        }
    }
    printf("  %s Results correct!\n", correct ? "✅" : "❌");
    
    // Step 6: Cleanup
    cudaFree(d_x);
    cudaFree(d_y);
    free(h_x);
    free(h_y);
    free(h_result);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
}

int main() {
    printf("=== CUDA Explicit Memory Management ===\n");
    
    // Get device info
    cudaDeviceProp prop;
    cudaGetDeviceProperties(&prop, 0);
    printf("Device: %s\n", prop.name);
    printf("Global Memory: %.1f GB\n", prop.totalGlobalMem / 1e9);
    
    // Test with different sizes
    explicitMemoryWorkflow(100000, 2.0f);      // 100K elements
    explicitMemoryWorkflow(1000000, 2.0f);     // 1M elements
    explicitMemoryWorkflow(10000000, 2.0f);    // 10M elements
    
    return 0;
}

In [ ]:
!nvcc -O3 -o explicit_memory explicit_memory.cu && ./explicit_memory

## 3. Unified Memory (cudaMallocManaged)

**Unified Memory** simplifies programming - memory is automatically migrated between CPU and GPU as needed.

```
┌──────────────┐                    ┌──────────────┐
│     CPU      │◄──── Automatic ────►│     GPU      │
│              │      Migration      │              │
│  ptr[i] = x  │                    │  ptr[i] = y  │
└──────────────┘                    └──────────────┘
        Same pointer works on both!
```

**Pros:**
- Simpler code - no explicit transfers
- Can access more data than GPU memory (oversubscription)

**Cons:**
- May have higher latency (page faults)
- Less control over when transfers happen

In [ ]:
%%writefile unified_memory.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

// Same SAXPY kernel works with unified memory!
__global__ void saxpy(float a, float* x, float* y, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        y[idx] = a * x[idx] + y[idx];
    }
}

void unifiedMemoryWorkflow(int n, float a) {
    printf("\n=== Unified Memory Workflow (N = %d) ===\n", n);
    printf("---------------------------------------------------\n");
    
    size_t size = n * sizeof(float);
    
    // Allocate UNIFIED memory - accessible from CPU and GPU!
    float *x, *y;
    cudaMallocManaged(&x, size);
    cudaMallocManaged(&y, size);
    
    // Initialize on CPU (data will migrate to GPU when needed)
    srand(42);
    for (int i = 0; i < n; i++) {
        x[i] = (float)rand() / RAND_MAX;
        y[i] = (float)rand() / RAND_MAX;
    }
    
    // Save original y for verification
    float *y_orig = (float*)malloc(size);
    for (int i = 0; i < n; i++) {
        y_orig[i] = y[i];
    }
    
    // Launch kernel - data migrates automatically!
    int threadsPerBlock = 256;
    int blocksPerGrid = (n + threadsPerBlock - 1) / threadsPerBlock;
    
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    cudaEventRecord(start);
    saxpy<<<blocksPerGrid, threadsPerBlock>>>(a, x, y, n);
    cudaDeviceSynchronize();  // Wait for kernel AND migration
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float totalTime;
    cudaEventElapsedTime(&totalTime, start, stop);
    printf("  Kernel + migration time: %.3f ms\n", totalTime);
    
    // Access result on CPU (data migrates back automatically)
    bool correct = true;
    for (int i = 0; i < n && correct; i++) {
        float expected = a * x[i] + y_orig[i];
        if (fabsf(y[i] - expected) > 1e-5) {
            printf("  Mismatch at %d: %f vs %f\n", i, y[i], expected);
            correct = false;
        }
    }
    printf("  %s Results correct!\n", correct ? "✅" : "❌");
    
    // Cleanup
    cudaFree(x);
    cudaFree(y);
    free(y_orig);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
}

int main() {
    printf("=== CUDA Unified Memory ===\n");
    
    cudaDeviceProp prop;
    cudaGetDeviceProperties(&prop, 0);
    printf("Device: %s\n", prop.name);
    printf("Managed Memory: %s\n", prop.managedMemory ? "Yes" : "No");
    
    // Test with different sizes
    unifiedMemoryWorkflow(100000, 2.0f);
    unifiedMemoryWorkflow(1000000, 2.0f);
    unifiedMemoryWorkflow(10000000, 2.0f);
    
    return 0;
}

In [ ]:
!nvcc -O3 -o unified_memory unified_memory.cu && ./unified_memory

## 4. Pinned (Page-Locked) Memory

Normal CPU memory can be **paged out** to disk by the OS. This causes problems for DMA transfers.

**Pinned memory** is locked in physical RAM - no paging allowed.

```
┌─────────────────────────────────────────────────────────────┐
│  Pageable Memory (default)     │  Pinned Memory             │
├─────────────────────────────────────────────────────────────┤
│  CPU Memory                    │  CPU Memory (locked)       │
│      ↓ (copy to staging)       │      ↓ (direct DMA)        │
│  Pinned Buffer                 │                            │
│      ↓ (DMA to GPU)            │      ↓ (DMA to GPU)        │
│  GPU Memory                    │  GPU Memory                │
├─────────────────────────────────────────────────────────────┤
│  TWO copies!                   │  ONE copy! (faster)        │
│  Can be swapped to disk        │  Always in RAM             │
│  Unlimited size                │  Limited by system RAM     │
└─────────────────────────────────────────────────────────────┘
```

In [ ]:
%%writefile pinned_memory.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

void comparePinnedVsPageable(int n) {
    printf("\n=== Pinned vs Pageable Memory (N = %d) ===\n", n);
    printf("---------------------------------------------------\n");
    
    size_t size = n * sizeof(float);
    
    // Allocate pageable memory (default malloc)
    float* h_pageable = (float*)malloc(size);
    
    // Allocate pinned (page-locked) memory
    float* h_pinned;
    cudaMallocHost(&h_pinned, size);  // or cudaHostAlloc
    
    // Initialize both with same data
    srand(42);
    for (int i = 0; i < n; i++) {
        float val = (float)rand() / RAND_MAX;
        h_pageable[i] = val;
        h_pinned[i] = val;
    }
    
    // Allocate device memory
    float* d_data;
    cudaMalloc(&d_data, size);
    
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    const int ITERATIONS = 10;
    
    // Benchmark pageable transfer
    cudaEventRecord(start);
    for (int i = 0; i < ITERATIONS; i++) {
        cudaMemcpy(d_data, h_pageable, size, cudaMemcpyHostToDevice);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float pageableTime;
    cudaEventElapsedTime(&pageableTime, start, stop);
    pageableTime /= ITERATIONS;
    
    // Benchmark pinned transfer
    cudaEventRecord(start);
    for (int i = 0; i < ITERATIONS; i++) {
        cudaMemcpy(d_data, h_pinned, size, cudaMemcpyHostToDevice);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float pinnedTime;
    cudaEventElapsedTime(&pinnedTime, start, stop);
    pinnedTime /= ITERATIONS;
    
    // Calculate bandwidth (GB/s)
    float dataSizeGB = size / 1e9;
    float pageableBW = dataSizeGB / (pageableTime / 1000);
    float pinnedBW = dataSizeGB / (pinnedTime / 1000);
    
    printf("  Pageable: %.3f ms (%.2f GB/s)\n", pageableTime, pageableBW);
    printf("  Pinned:   %.3f ms (%.2f GB/s)\n", pinnedTime, pinnedBW);
    printf("  Speedup:  %.2fx\n", pageableTime / pinnedTime);
    
    // Cleanup
    cudaFree(d_data);
    cudaFreeHost(h_pinned);
    free(h_pageable);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
}

int main() {
    printf("=== CUDA Pinned Memory ===\n");
    
    cudaDeviceProp prop;
    cudaGetDeviceProperties(&prop, 0);
    printf("Device: %s\n", prop.name);
    
    // Test with different sizes
    comparePinnedVsPageable(1000000);     // 1M elements (4 MB)
    comparePinnedVsPageable(10000000);    // 10M elements (40 MB)
    comparePinnedVsPageable(50000000);    // 50M elements (200 MB)
    
    printf("\n💡 Tip: Use pinned memory for frequent CPU-GPU transfers!\n");
    printf("   But don't over-allocate - it reduces available RAM.\n");
    
    return 0;
}

In [ ]:
!nvcc -O3 -o pinned_memory pinned_memory.cu && ./pinned_memory

## 5. When Transfer Time Dominates

For simple operations like vector addition, **memory transfers dominate execution time**.

This is why:
1. GPU is best for **compute-intensive** operations
2. You should **minimize transfers** (keep data on GPU)
3. **Batch operations** together before copying back

In [ ]:
%%writefile transfer_analysis.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

__global__ void addKernel(float* a, float* b, float* c, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        c[idx] = a[idx] + b[idx];
    }
}

void analyzeTransferOverhead(int n) {
    printf("\n=== Transfer vs Compute Analysis (N = %d) ===\n", n);
    printf("------------------------------------------------------------\n");
    
    size_t size = n * sizeof(float);
    
    // Allocate host memory
    float *h_a = (float*)malloc(size);
    float *h_b = (float*)malloc(size);
    float *h_c = (float*)malloc(size);
    
    // Initialize
    for (int i = 0; i < n; i++) {
        h_a[i] = 1.0f;
        h_b[i] = 2.0f;
    }
    
    // Allocate device memory
    float *d_a, *d_b, *d_c;
    cudaMalloc(&d_a, size);
    cudaMalloc(&d_b, size);
    cudaMalloc(&d_c, size);
    
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    // Measure H2D transfer
    cudaEventRecord(start);
    cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, size, cudaMemcpyHostToDevice);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float h2dTime;
    cudaEventElapsedTime(&h2dTime, start, stop);
    
    // Measure kernel (warmup first)
    int threads = 256;
    int blocks = (n + threads - 1) / threads;
    addKernel<<<blocks, threads>>>(d_a, d_b, d_c, n);
    cudaDeviceSynchronize();
    
    cudaEventRecord(start);
    for (int i = 0; i < 100; i++) {
        addKernel<<<blocks, threads>>>(d_a, d_b, d_c, n);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float kernelTime;
    cudaEventElapsedTime(&kernelTime, start, stop);
    kernelTime /= 100;
    
    // Measure D2H transfer
    cudaEventRecord(start);
    cudaMemcpy(h_c, d_c, size, cudaMemcpyDeviceToHost);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float d2hTime;
    cudaEventElapsedTime(&d2hTime, start, stop);
    
    float total = h2dTime + kernelTime + d2hTime;
    
    printf("  H2D Transfer:  %8.3f ms (%5.1f%%)\n", h2dTime, h2dTime/total*100);
    printf("  Kernel:        %8.3f ms (%5.1f%%)\n", kernelTime, kernelTime/total*100);
    printf("  D2H Transfer:  %8.3f ms (%5.1f%%)\n", d2hTime, d2hTime/total*100);
    printf("  ─────────────────────────────────\n");
    printf("  Total:         %8.3f ms\n", total);
    
    if ((h2dTime + d2hTime) > kernelTime * 10) {
        printf("\n  ⚠️  Transfer time dominates!\n");
        printf("     Consider: Keep data on GPU, batch operations!\n");
    }
    
    // Cleanup
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    free(h_a);
    free(h_b);
    free(h_c);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
}

int main() {
    printf("=== CUDA Transfer Overhead Analysis ===\n");
    
    analyzeTransferOverhead(1000000);    // 1M elements
    analyzeTransferOverhead(10000000);   // 10M elements
    
    return 0;
}

In [ ]:
!nvcc -O3 -o transfer_analysis transfer_analysis.cu && ./transfer_analysis

## 6. Memory Reuse Pattern

**Best Practice:** Keep data on GPU for multiple operations!

In [ ]:
%%writefile memory_reuse.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

// Multiple operations - all stay on GPU
__global__ void addKernel(float* a, float* b, float* out, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) out[idx] = a[idx] + b[idx];
}

__global__ void mulKernel(float* a, float* b, float* out, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) out[idx] = a[idx] * b[idx];
}

__global__ void subKernel(float* a, float* b, float* out, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) out[idx] = a[idx] - b[idx];
}

int main() {
    printf("=== Memory Reuse Pattern ===\n\n");
    printf("Computing: result = ((a + b) * c) - d\n");
    printf("Using memory reuse to minimize transfers!\n\n");
    
    const int N = 10000000;  // 10M elements
    size_t size = N * sizeof(float);
    
    // Allocate and initialize host data
    float *h_a = (float*)malloc(size);
    float *h_b = (float*)malloc(size);
    float *h_c = (float*)malloc(size);
    float *h_d = (float*)malloc(size);
    float *h_result = (float*)malloc(size);
    
    for (int i = 0; i < N; i++) {
        h_a[i] = 1.0f;
        h_b[i] = 2.0f;
        h_c[i] = 3.0f;
        h_d[i] = 4.0f;
    }
    
    // Allocate device memory
    float *d_a, *d_b, *d_c, *d_d, *d_temp1, *d_temp2, *d_result;
    cudaMalloc(&d_a, size);
    cudaMalloc(&d_b, size);
    cudaMalloc(&d_c, size);
    cudaMalloc(&d_d, size);
    cudaMalloc(&d_temp1, size);
    cudaMalloc(&d_temp2, size);
    cudaMalloc(&d_result, size);
    
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    int threads = 256;
    int blocks = (N + threads - 1) / threads;
    
    cudaEventRecord(start);
    
    // Single H2D transfer - copy all inputs ONCE
    cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_c, h_c, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_d, h_d, size, cudaMemcpyHostToDevice);
    
    // All computation stays on GPU!
    addKernel<<<blocks, threads>>>(d_a, d_b, d_temp1, N);    // temp1 = a + b
    mulKernel<<<blocks, threads>>>(d_temp1, d_c, d_temp2, N); // temp2 = temp1 * c
    subKernel<<<blocks, threads>>>(d_temp2, d_d, d_result, N);// result = temp2 - d
    
    // Single D2H transfer - copy only result
    cudaMemcpy(h_result, d_result, size, cudaMemcpyDeviceToHost);
    
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float totalTime;
    cudaEventElapsedTime(&totalTime, start, stop);
    
    // Verify
    bool correct = true;
    for (int i = 0; i < N && correct; i++) {
        float expected = ((h_a[i] + h_b[i]) * h_c[i]) - h_d[i];
        if (fabsf(h_result[i] - expected) > 1e-5) {
            printf("Mismatch at %d: %f vs %f\n", i, h_result[i], expected);
            correct = false;
        }
    }
    
    printf("Pipeline executed in: %.3f ms\n", totalTime);
    printf("%s Results correct!\n", correct ? "✅" : "❌");
    printf("\n💡 Only 5 transfers (4 in + 1 out) instead of 6+!\n");
    
    // Cleanup
    cudaFree(d_a); cudaFree(d_b); cudaFree(d_c); cudaFree(d_d);
    cudaFree(d_temp1); cudaFree(d_temp2); cudaFree(d_result);
    free(h_a); free(h_b); free(h_c); free(h_d); free(h_result);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    
    return 0;
}

In [ ]:
!nvcc -O3 -o memory_reuse memory_reuse.cu && ./memory_reuse

## 🎯 Exercises

### Exercise 1: Compare All Memory Types
Complete this program to compare explicit, unified, and pinned memory.

In [ ]:
%%writefile exercise1_compare_memory.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

__global__ void scaleKernel(float* data, float scale, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        data[idx] *= scale;
    }
}

// TODO: Implement these three functions
float testExplicitMemory(int n) {
    // 1. Allocate with malloc and cudaMalloc
    // 2. cudaMemcpy H2D
    // 3. Launch kernel
    // 4. cudaMemcpy D2H
    // 5. Return total time
    return 0.0f;
}

float testUnifiedMemory(int n) {
    // 1. Allocate with cudaMallocManaged
    // 2. Initialize on CPU
    // 3. Launch kernel + synchronize
    // 4. Return total time
    return 0.0f;
}

float testPinnedMemory(int n) {
    // 1. Allocate with cudaMallocHost
    // 2. cudaMemcpy H2D
    // 3. Launch kernel
    // 4. cudaMemcpy D2H
    // 5. Return total time
    return 0.0f;
}

int main() {
    printf("Exercise 1: Compare Memory Types!\n");
    printf("Implement the three test functions above.\n");
    return 0;
}

## 📝 Key Takeaways

### Memory Management Summary:

| Approach | API | Pros | Cons | Use When |
|----------|-----|------|------|----------|
| **Explicit** | `cudaMalloc` / `cudaMemcpy` | Full control, predictable | Verbose, error-prone | Performance critical |
| **Unified** | `cudaMallocManaged` | Simple, automatic | Less control, page faults | Prototyping, complex access |
| **Pinned** | `cudaMallocHost` | Faster transfers | Uses locked RAM | Frequent CPU-GPU transfers |

### Best Practices:

1. **Minimize transfers** - Keep data on GPU for multiple operations
2. **Use pinned memory** for frequently transferred data
3. **Overlap transfers and compute** using streams (Week 5)
4. **Profile first** - Measure where time is actually spent

---

### 📚 Next Up: Day 4 - Error Handling & Debugging
- The CUDA_CHECK macro
- Error codes and their meanings
- Debugging with compute-sanitizer

---

### 🔗 Resources
- [Memory Management - Programming Guide](../../cuda-programming-guide/02-basics/cuda-memory.md)
- [Quick Reference](../../notes/cuda-quick-reference.md)

In [ ]:
# Cleanup generated files
!rm -f explicit_memory unified_memory pinned_memory transfer_analysis memory_reuse
!rm -f *.cu
print("✅ Cleanup complete!")

In [ ]:
# ⚙️ Colab/Local Setup - Run this first!
# Python/Numba is OPTIONAL - for quick interactive testing only
import subprocess, sys
try:
    import google.colab
    print("🔧 Running on Google Colab - Installing dependencies...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "numba"])
    print("✅ Setup complete!")
except ImportError:
    print("💻 Running locally - make sure you have: pip install numba numpy")

import numpy as np
from numba import cuda
import math
import time

print("\n⚠️  Remember: CUDA C++ code is the PRIMARY learning material!")
print("   Python/Numba is provided for quick interactive testing only.")

# Day 3: GPU Memory Fundamentals

Memory management is **the most important skill** in CUDA programming. Today you'll learn:
- GPU memory hierarchy
- Explicit memory management (`cudaMalloc`, `cudaMemcpy`, `cudaFree`)
- Unified/Managed memory (`cudaMallocManaged`)
- Pinned vs pageable memory for transfers
- Performance implications of each approach

---

## 1. The Memory Hierarchy

```
┌─────────────────────────────────────────────────────────────────┐
│                        GPU MEMORY HIERARCHY                      │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│   Registers (per thread)     ← Fastest (~0 latency)             │
│   ├── ~255 32-bit registers per thread                          │
│   └── Private to each thread                                    │
│                                                                  │
│   Shared Memory (per block)  ← Very fast (~5 cycles)            │
│   ├── 48-164 KB per SM                                          │
│   └── Shared between threads in same block                      │
│                                                                  │
│   L1 Cache / Texture Cache   ← Fast (~30 cycles)                │
│   └── Automatic caching                                         │
│                                                                  │
│   L2 Cache                   ← Medium (~200 cycles)             │
│   └── Shared across all SMs                                     │
│                                                                  │
│   Global Memory (VRAM)       ← Slow (~400+ cycles)              │
│   ├── 4-80 GB (main GPU memory)                                 │
│   └── Accessible by all threads                                 │
│                                                                  │
├─────────────────────────────────────────────────────────────────┤
│   Host Memory (System RAM)   ← Very slow (PCIe transfer)        │
│   └── Must be copied to GPU before use                          │
└─────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Setup
import numpy as np
from numba import cuda
import math
import time

print("CUDA device:", cuda.get_current_device().name.decode())

# Get memory info
ctx = cuda.current_context()
free_mem, total_mem = ctx.get_memory_info()
print(f"GPU Memory: {free_mem/1e9:.2f} GB free / {total_mem/1e9:.2f} GB total")

## 2. Explicit Memory Management

The traditional CUDA workflow requires **explicit** memory transfers:

```
1. Allocate memory on GPU (cudaMalloc)
2. Copy data from CPU to GPU (cudaMemcpy H2D)
3. Launch kernel
4. Copy results from GPU to CPU (cudaMemcpy D2H)
5. Free GPU memory (cudaFree)
```

In Numba:
- `cuda.to_device(array)` - Allocates and copies to GPU
- `cuda.device_array(shape, dtype)` - Allocates on GPU only
- `device_array.copy_to_host()` - Copies back to CPU

In [ ]:
@cuda.jit
def saxpy_kernel(a, x, y, out, n):
    """SAXPY: out = a*x + y (Single-precision A*X Plus Y)"""
    idx = cuda.grid(1)
    if idx < n:
        out[idx] = a * x[idx] + y[idx]

def explicit_memory_workflow(n, a=2.0):
    """Demonstrate explicit memory management"""
    print(f"\n📊 Explicit Memory Workflow (N = {n:,})")
    print("-" * 50)
    
    # Step 1: Create data on CPU
    x_host = np.random.randn(n).astype(np.float32)
    y_host = np.random.randn(n).astype(np.float32)
    out_host = np.zeros(n, dtype=np.float32)
    
    # Step 2: Allocate and copy to GPU
    start = time.perf_counter()
    x_device = cuda.to_device(x_host)        # Allocate + copy
    y_device = cuda.to_device(y_host)        # Allocate + copy
    out_device = cuda.device_array(n, dtype=np.float32)  # Allocate only
    h2d_time = time.perf_counter() - start
    print(f"  Host→Device transfer: {h2d_time*1000:.3f} ms")
    
    # Step 3: Launch kernel
    threads = 256
    blocks = math.ceil(n / threads)
    
    start = time.perf_counter()
    saxpy_kernel[blocks, threads](a, x_device, y_device, out_device, n)
    cuda.synchronize()
    kernel_time = time.perf_counter() - start
    print(f"  Kernel execution:      {kernel_time*1000:.3f} ms")
    
    # Step 4: Copy result back
    start = time.perf_counter()
    out_host = out_device.copy_to_host()
    d2h_time = time.perf_counter() - start
    print(f"  Device→Host transfer:  {d2h_time*1000:.3f} ms")
    
    print(f"  Total time:            {(h2d_time + kernel_time + d2h_time)*1000:.3f} ms")
    
    # Verify
    expected = a * x_host + y_host
    print(f"  ✅ Correct: {np.allclose(out_host, expected)}")
    
    return h2d_time, kernel_time, d2h_time

# Run with different sizes
for size in [100_000, 1_000_000, 10_000_000]:
    explicit_memory_workflow(size)

## 3. Unified Memory (cudaMallocManaged)

**Unified Memory** simplifies programming - memory is automatically migrated between CPU and GPU as needed.

```
┌──────────────┐                    ┌──────────────┐
│     CPU      │◄──── Automatic ────►│     GPU      │
│              │      Migration      │              │
│  ptr[i] = x  │                    │  ptr[i] = y  │
└──────────────┘                    └──────────────┘
        Same pointer works on both!
```

**Pros:**
- Simpler code
- No explicit transfers needed
- Can access more data than GPU memory (oversubscription)

**Cons:**
- May have higher latency (page faults)
- Less control over when transfers happen
- Can be slower if not used carefully

In [ ]:
def unified_memory_workflow(n, a=2.0):
    """Demonstrate unified memory (managed memory)"""
    print(f"\n📊 Unified Memory Workflow (N = {n:,})")
    print("-" * 50)
    
    # Allocate managed memory - accessible from both CPU and GPU
    x = cuda.managed_array(n, dtype=np.float32)
    y = cuda.managed_array(n, dtype=np.float32)
    out = cuda.managed_array(n, dtype=np.float32)
    
    # Initialize on CPU (memory will migrate to GPU when needed)
    x[:] = np.random.randn(n).astype(np.float32)
    y[:] = np.random.randn(n).astype(np.float32)
    
    # Launch kernel (data migrates automatically)
    threads = 256
    blocks = math.ceil(n / threads)
    
    start = time.perf_counter()
    saxpy_kernel[blocks, threads](a, x, y, out, n)
    cuda.synchronize()
    total_time = time.perf_counter() - start
    print(f"  Kernel + migration time: {total_time*1000:.3f} ms")
    
    # Access result on CPU (data migrates back automatically)
    expected = a * x + y
    print(f"  ✅ Correct: {np.allclose(out, expected)}")
    
    return total_time

# Compare with explicit management
for size in [100_000, 1_000_000, 10_000_000]:
    unified_memory_workflow(size)

## 4. Pinned (Page-Locked) Memory

Normal CPU memory can be **paged out** to disk by the OS. This causes problems for DMA transfers.

**Pinned memory** is locked in physical RAM - no paging allowed.

```
┌─────────────────────────────────────────────────────────────┐
│  Pageable Memory (default)     │  Pinned Memory             │
├─────────────────────────────────────────────────────────────┤
│  CPU Memory                    │  CPU Memory (locked)       │
│      ↓ (copy to staging)       │      ↓ (direct DMA)        │
│  Pinned Buffer                 │                            │
│      ↓ (DMA to GPU)            │      ↓ (DMA to GPU)        │
│  GPU Memory                    │  GPU Memory                │
├─────────────────────────────────────────────────────────────┤
│  TWO copies!                   │  ONE copy! (faster)        │
│  Can be swapped to disk        │  Always in RAM             │
│  Unlimited size                │  Limited by system RAM     │
└─────────────────────────────────────────────────────────────┘
```

In [ ]:
def compare_pinned_vs_pageable(n):
    """Compare transfer speeds with pinned vs pageable memory"""
    print(f"\n📊 Pinned vs Pageable Memory (N = {n:,})")
    print("-" * 50)
    
    # Pageable memory (default NumPy allocation)
    pageable = np.random.randn(n).astype(np.float32)
    
    # Pinned memory
    pinned = cuda.pinned_array(n, dtype=np.float32)
    pinned[:] = pageable  # Copy data to pinned
    
    # Measure pageable transfer
    start = time.perf_counter()
    for _ in range(10):
        d_pageable = cuda.to_device(pageable)
        cuda.synchronize()
    pageable_time = (time.perf_counter() - start) / 10
    
    # Measure pinned transfer
    start = time.perf_counter()
    for _ in range(10):
        d_pinned = cuda.to_device(pinned)
        cuda.synchronize()
    pinned_time = (time.perf_counter() - start) / 10
    
    # Calculate bandwidth
    data_size_gb = n * 4 / 1e9  # float32 = 4 bytes
    pageable_bw = data_size_gb / pageable_time
    pinned_bw = data_size_gb / pinned_time
    
    print(f"  Pageable: {pageable_time*1000:.3f} ms ({pageable_bw:.2f} GB/s)")
    print(f"  Pinned:   {pinned_time*1000:.3f} ms ({pinned_bw:.2f} GB/s)")
    print(f"  Speedup:  {pageable_time/pinned_time:.2f}x")

# Compare for different sizes
for size in [1_000_000, 10_000_000, 50_000_000]:
    compare_pinned_vs_pageable(size)

## 5. When Transfer Time Dominates

For simple operations like vector addition, **memory transfers dominate execution time**.

This is why:
1. GPU is best for **compute-intensive** operations
2. You should **minimize transfers** (keep data on GPU)
3. **Batch operations** together before copying back

In [ ]:
def analyze_transfer_overhead(n):
    """Show how transfer time compares to compute time"""
    print(f"\n📊 Transfer vs Compute Analysis (N = {n:,})")
    print("-" * 60)
    
    a = np.random.randn(n).astype(np.float32)
    b = np.random.randn(n).astype(np.float32)
    
    # Measure H2D transfer
    start = time.perf_counter()
    a_d = cuda.to_device(a)
    b_d = cuda.to_device(b)
    c_d = cuda.device_array(n, dtype=np.float32)
    cuda.synchronize()
    h2d_time = time.perf_counter() - start
    
    # Measure kernel (data already on GPU)
    threads = 256
    blocks = math.ceil(n / threads)
    
    @cuda.jit
    def add_kernel(a, b, c):
        idx = cuda.grid(1)
        if idx < c.size:
            c[idx] = a[idx] + b[idx]
    
    # Warmup
    add_kernel[blocks, threads](a_d, b_d, c_d)
    cuda.synchronize()
    
    start = time.perf_counter()
    for _ in range(100):
        add_kernel[blocks, threads](a_d, b_d, c_d)
    cuda.synchronize()
    kernel_time = (time.perf_counter() - start) / 100
    
    # Measure D2H transfer
    start = time.perf_counter()
    c = c_d.copy_to_host()
    cuda.synchronize()
    d2h_time = time.perf_counter() - start
    
    total = h2d_time + kernel_time + d2h_time
    
    print(f"  H2D Transfer:  {h2d_time*1000:>8.3f} ms ({h2d_time/total*100:>5.1f}%)")
    print(f"  Kernel:        {kernel_time*1000:>8.3f} ms ({kernel_time/total*100:>5.1f}%)")
    print(f"  D2H Transfer:  {d2h_time*1000:>8.3f} ms ({d2h_time/total*100:>5.1f}%)")
    print(f"  ─────────────────────────────────────")
    print(f"  Total:         {total*1000:>8.3f} ms")
    
    if (h2d_time + d2h_time) > kernel_time:
        print(f"\n  ⚠️  Transfer time ({(h2d_time + d2h_time)*1000:.2f} ms) > Kernel time ({kernel_time*1000:.2f} ms)")
        print(f"     Consider: Keep data on GPU, batch operations!")

analyze_transfer_overhead(10_000_000)

## 🎯 Exercises

### Exercise 1: Memory Reuse Pattern
Implement a pipeline that reuses GPU memory for multiple operations without copying back to CPU between each step.

In [ ]:
# TODO Exercise 1: Memory Reuse Pattern
# Compute: result = ((a + b) * c) - d
# Do this with ONE H2D transfer and ONE D2H transfer

@cuda.jit
def add_kernel(a, b, out):
    idx = cuda.grid(1)
    if idx < out.size:
        out[idx] = a[idx] + b[idx]

@cuda.jit
def mul_kernel(a, b, out):
    idx = cuda.grid(1)
    if idx < out.size:
        out[idx] = a[idx] * b[idx]

@cuda.jit  
def sub_kernel(a, b, out):
    idx = cuda.grid(1)
    if idx < out.size:
        out[idx] = a[idx] - b[idx]

def pipeline_inefficient(a, b, c, d):
    """BAD: Transfers after each operation"""
    # TODO: This is the slow way - see how many transfers happen
    pass

def pipeline_efficient(a, b, c, d):
    """GOOD: All operations on GPU, one final transfer"""
    # TODO: Implement efficient version
    # 1. Copy all inputs to GPU
    # 2. Perform all operations
    # 3. Copy only result back
    pass

# Test your implementations
# N = 10_000_000
# a = np.random.randn(N).astype(np.float32)
# ...

### Exercise 2: Memory Bandwidth Calculation
Calculate the theoretical vs achieved memory bandwidth for a copy operation.

In [ ]:
# TODO Exercise 2: Bandwidth Measurement
@cuda.jit
def copy_kernel(src, dst):
    """Simple copy kernel - measures GPU memory bandwidth"""
    idx = cuda.grid(1)
    if idx < dst.size:
        dst[idx] = src[idx]

def measure_bandwidth(n):
    """Measure achieved memory bandwidth"""
    # TODO: 
    # 1. Create two device arrays
    # 2. Time the copy kernel
    # 3. Calculate bandwidth: (bytes_read + bytes_written) / time
    # 4. Compare to theoretical peak (from device properties)
    pass

# measure_bandwidth(100_000_000)

## 📝 Key Takeaways

### Memory Management Summary:

| Method | When to Use | Pros | Cons |
|--------|-------------|------|------|
| **Explicit** (`to_device`, `copy_to_host`) | Production code, max control | Fastest, predictable | More code |
| **Unified** (`managed_array`) | Prototyping, complex access patterns | Simple code | Page fault overhead |
| **Pinned** (`pinned_array`) | High-bandwidth transfers | ~2x transfer speed | Uses system RAM |

### Best Practices:

1. **Minimize transfers**: Keep data on GPU as long as possible
2. **Use pinned memory**: For frequent large transfers
3. **Batch operations**: Don't copy back between every operation
4. **Profile first**: Measure before optimizing

### Memory Bandwidth Rule of Thumb:
- PCIe 3.0 x16: ~16 GB/s
- PCIe 4.0 x16: ~32 GB/s  
- GPU Memory: 200-900 GB/s (much faster!)

---

### 📚 Next Up: Day 4 - Error Handling & Debugging
- CUDA error codes
- Debugging techniques
- Common pitfalls

---

### 🔗 Resources
- [CUDA Memory Guide](../../cuda-programming-guide/02-basics/cuda-memory.md)
- [Understanding Memory](../../cuda-programming-guide/02-basics/understanding-memory.md)